# Graph-Enriched Context Provider

In this notebook, you'll combine **vector search** with **graph traversal** to provide rich, structured context. Instead of just returning matched text chunks, the provider will traverse relationships to include company names, products, and risk factors.

***

Load the environment variables and import the required modules.

In [ ]:
import sys
sys.path.insert(0, '../shared')

from azure.identity import DefaultAzureCredential
from azure.identity.aio import AzureCliCredential

from agent_framework.azure import AzureAIClient
from agent_framework_neo4j import (
    AzureAIEmbedder,
    AzureAISettings,
    Neo4jContextProvider,
    Neo4jSettings,
)

from config import get_agent_config

Get the configuration and create the embedder.

In [ ]:
# Get configuration from environment
config = get_agent_config()
neo4j_settings = Neo4jSettings()
azure_settings = AzureAISettings()

# Create embedder
sync_credential = DefaultAzureCredential()

embedder = AzureAIEmbedder(
    endpoint=azure_settings.inference_endpoint,
    credential=sync_credential,
    model=azure_settings.embedding_model,
)

## Define the Retrieval Query

The retrieval query is a Cypher query that traverses the graph after vector search finds matching chunks. It receives `node` (the matched Chunk) and `score` (similarity score) from the vector search.

This query traverses:
- `Chunk` → `Document` → `Company` (who filed the document)
- `Company` → `RiskFactor` (risks the company faces)
- `Company` → `Product` (products the company mentions)

In [ ]:
# Graph-enriched retrieval query
# Appended after vector search: YIELD node, score
RETRIEVAL_QUERY = """
MATCH (node)-[:FROM_DOCUMENT]->(doc:Document)<-[:FILED]-(company:Company)
OPTIONAL MATCH (company)-[:FACES_RISK]->(risk:RiskFactor)
WITH node, score, company, doc,
     collect(DISTINCT risk.name)[0..5] AS risks
OPTIONAL MATCH (company)-[:MENTIONS]->(product:Product)
WITH node, score, company, doc, risks,
     collect(DISTINCT product.name)[0..5] AS products
WHERE score IS NOT NULL
RETURN
    node.text AS text,
    score,
    company.name AS company,
    company.ticker AS ticker,
    risks,
    products
ORDER BY score DESC
"""

print("Retrieval Query Pattern:")
print("  Chunk -[:FROM_DOCUMENT]-> Document <-[:FILED]- Company")
print("  Company -[:FACES_RISK]-> RiskFactor")
print("  Company -[:MENTIONS]-> Product")

> The retrieval query must use the `node` and `score` variables provided by the vector search. It must return at least `text` and `score` columns.

***

## Create the Graph-Enriched Context Provider

When a `retrieval_query` is provided, the provider internally uses `VectorCypherRetriever` instead of the basic `VectorRetriever`. Results include both the matched text and traversed graph context.

In [ ]:
# Create context provider with graph-enriched mode
provider = Neo4jContextProvider(
    uri=neo4j_settings.uri,
    username=neo4j_settings.username,
    password=neo4j_settings.get_password(),
    index_name=neo4j_settings.vector_index_name,
    index_type="vector",
    retrieval_query=RETRIEVAL_QUERY,
    embedder=embedder,
    top_k=5,
    context_prompt=(
        "## Graph-Enriched Knowledge Context\n"
        "The following information combines semantic search results with "
        "graph traversal to provide company, product, and risk context:"
    ),
)

> The key difference from the previous notebook is the `retrieval_query` parameter. This tells the provider to use graph traversal for enriched results.

***

## Run the Agent with Graph-Enriched Context

The agent now receives context that includes company names, products, and risk factors alongside the matched text. The agent is created once and reused for all queries below.

In [ ]:
# Connect provider and create agent
credential = AzureCliCredential()
await provider.__aenter__()
print("Connected to Neo4j with graph-enriched mode!\n")

client = AzureAIClient(
    project_endpoint=config.project_endpoint,
    model_deployment_name=config.model_name,
    async_credential=credential,
)
await client.__aenter__()

agent = await client.create_agent(
    name="workshop-graph-enriched-agent",
    instructions=(
        "You are a helpful assistant that answers questions about companies "
        "using graph-enriched context. Your context includes:\n"
        "- Semantic search matches from company filings\n"
        "- Company names and ticker symbols\n"
        "- Products the company mentions\n"
        "- Risk factors the company faces\n\n"
        "When answering, cite the company, relevant products, and risks. "
        "Be specific and reference the enriched graph data."
    ),
    context_providers=[provider],
).__aenter__()

session = agent.create_session()

async def ask(query):
    """Run a query against the agent and print the response."""
    print(f"User: {query}\n")
    print("Assistant: ", end="", flush=True)
    response = await agent.run(query, session=session)
    print(response.text)
    print()

print(f"Agent '{agent.name}' created")

In [ ]:
await ask("What are Apple's main products and what risks does the company face?")

> Graph enrichment provides structured metadata (company, ticker, risks, products) alongside the text, enabling more specific and accurate responses.

***

Run the cells below to experiment with different queries. You can also modify the queries or add new cells.

***

[View the complete code](../financial_data_load/solution_srcs/04_03_graph_enriched_provider.py)

[Move on to the Fulltext Context Provider Notebook (Optional)](03_fulltext_context_provider.ipynb)

In [ ]:
await ask("Tell me about Microsoft's cloud services and business risks")

In [ ]:
await ask("What products and risks are mentioned in Amazon's filings?")

In [ ]:
# Cleanup
await agent.__aexit__(None, None, None)
await client.__aexit__(None, None, None)
await provider.__aexit__(None, None, None)
await credential.close()
embedder.close()